In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')


In [ ]:
all_food = pd.concat([one, two, three, four, top_rest], axis =0)

In [ ]:
all_food.info()

In [ ]:
def to_city(df):
    city_df = df.groupby(['country','city', 'id', 'price_level'])['rating'].mean().to_frame()
    city_df['restaurant_count'] = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count()
    city_df['price_average_all'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()


    return city_df

In [ ]:
test = to_city(all_food)

In [ ]:
test

In [ ]:
test.loc['Thailand']

In [ ]:
describe_df = all_food.groupby(['country', 'city', 'price_level'])['rating'].describe()

In [ ]:
describe_df

In [ ]:
attractions = get_df('attractions')
attractions.sample(5)

In [ ]:
place_of_worship = ['place_of_worship', 'hindu_temple', 'church', 'mosque', 'synagogue']
shopping = ['store', 'shopping_mall', 'clothing_store', 'electronics_store', 'grocery_or_supermarket', 'department_store']

attractions_to_keep = ['amusement_park', 'museum', 'park', 'art_gallery', 'aquarium',
                      'zoo', 'library', 'movie_theater', 'natural_feature'] + place_of_worship + shopping

attractions_to_keep


In [ ]:

def split_types(df):
    df['split_types'] = [ast.literal_eval(x) for x in df.types]
    df['split_types_str'] = [','.join(x) for x in df.split_types]
    
    return df

def dummies(df):
    dummies = df.split_types_str.str.get_dummies(sep=',')

    return dummies

def attraction_count(dummies_df, all_attractions_df):
    all_attractions_df = pd.concat([all_attractions_df, dummies_df], axis=1)

    #type_col_names = dummies_df.columns.tolist()
    type_col_names = attractions_to_keep
    type_col_names.extend(['country', 'city', 'id'])
    attraction_count = all_attractions_df[type_col_names].groupby(['country', 'city', 'id']).sum()
    
    return attraction_count, all_attractions_df

In [ ]:
attractions_split = split_types(attractions)
attractions_split

In [ ]:
df = dummies(attractions_split)
df

In [ ]:
df2, all_attractions = attraction_count(df, attractions_split)

In [ ]:
df2

In [ ]:
tourist_types = attractions_split['split_types'].apply(pd.Series).stack().unique()
len(tourist_types)

In [ ]:
for t in tourist_types:
    print(t)

In [ ]:
all_attractions

In [ ]:
left_out = ['country', 'city','school', 'stadium', 'gym', 'casino', 'bowling_alley', 'night_club',
           'spa', 'beauty_salon', 'home_goods_store', 'book_store', 'university', 'campground']

In [ ]:
df2['left_out'] =  all_attractions[left_out].groupby(['country', 'city']).sum().sum(axis=1)


In [ ]:
df2['natural_feature'].sum()

In [ ]:
test

In [ ]:
city = pd.merge(df2, test, on = ['country', 'city', 'id'], how = 'outer')

In [ ]:
city.sort_values(['id', 'city', 'country'])

In [ ]:
city[city['rating'].isnull()]

In [ ]:
city.fillna(0, inplace=True)

In [ ]:
cosine_city = pd.DataFrame(cosine_similarity(city))

cosine_city

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
top_10 = find_similar_n(cosine_city, 10)
top_10

In [ ]:
city.iloc[[0,68,69,344,66,76,318,347,382,354]]

In [ ]:
city.iloc[[1, 2,437,436,480,481,493,444,445,22]]

In [ ]:
city.iloc[[510,241,511,242,243,145,406,193,388,349]]